<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/GPAW_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run GPAW on Google-colab and calculate interface energy with jarvis-tools

In [6]:
!apt install python3-mpi4py cython3 libxc-dev gpaw-data
!pip -q install gpaw jarvis-tools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cython3 is already the newest version (0.29.28-1ubuntu3).
gpaw-data is already the newest version (0.9.20000-2).
libxc-dev is already the newest version (5.1.7-1ubuntu1).
python3-mpi4py is already the newest version (3.1.3-1build2).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 10.2 MB/s eta 0:00:00


In [2]:
!gpaw test

 -------------------------------------------------------------------------------------------------
| python-3.10.12    /usr/bin/python3                                                              |
| gpaw-23.6.1       /usr/local/lib/python3.10/dist-packages/gpaw/                                 |
| ase-3.22.1        /usr/local/lib/python3.10/dist-packages/ase/                                  |
| numpy-1.22.4      /usr/local/lib/python3.10/dist-packages/numpy/                                |
| scipy-1.10.1      /usr/local/lib/python3.10/dist-packages/scipy/                                |
| libxc-5.1.7       yes                                                                           |
| _gpaw             /usr/local/lib/python3.10/dist-packages/_gpaw.cpython-310-x86_64-linux-gnu.so |
| MPI enabled       yes                                                                           |
| OpenMP enabled    no                                                                            |
|

In [3]:
!gpaw info

 -------------------------------------------------------------------------------------------------
| python-3.10.12    /usr/bin/python3                                                              |
| gpaw-23.6.1       /usr/local/lib/python3.10/dist-packages/gpaw/                                 |
| ase-3.22.1        /usr/local/lib/python3.10/dist-packages/ase/                                  |
| numpy-1.22.4      /usr/local/lib/python3.10/dist-packages/numpy/                                |
| scipy-1.10.1      /usr/local/lib/python3.10/dist-packages/scipy/                                |
| libxc-5.1.7       yes                                                                           |
| _gpaw             /usr/local/lib/python3.10/dist-packages/_gpaw.cpython-310-x86_64-linux-gnu.so |
| MPI enabled       yes                                                                           |
| OpenMP enabled    no                                                                            |
|

FCC Aluminum

In [4]:
# Sample calculation from https://wiki.fysik.dtu.dk/gpaw/tutorialsexercises/structureoptimization/aluminium/aluminium.html
from ase import Atoms
from ase.visualize import view
from gpaw import GPAW, PW

name = 'Al-fcc'
a = 4.05  # fcc lattice parameter
b = a / 2

bulk = Atoms('Al',
             cell=[[0, b, b],
                   [b, 0, b],
                   [b, b, 0]],
             pbc=True)

view(bulk)

k = 4
calc = GPAW(mode=PW(300),       # cutoff
            kpts=(k, k, k),     # k-points
            txt=name + '.txt')  # output file

bulk.calc = calc

energy = bulk.get_potential_energy()
calc.write(name + '.gpw')
print('Energy:', energy, 'eV')

Energy: -4.122151616078365 eV


In [7]:
from jarvis.db.figshare import get_jid_data
from jarvis.core.atoms import Atoms
from jarvis.core.kpoints import Kpoints3D
jid = 'JVASP-816'
dat = get_jid_data(jid=jid,dataset='dft_3d')
atoms = Atoms.from_dict(dat['atoms'])
atoms = atoms.get_conventional_atoms
ase_atoms = (atoms.get_conventional_atoms).ase_converter()
kp = Kpoints3D().automatic_length_mesh(
    #lattice_mat=atoms.lattice_mat,
    #length=10
    lattice_mat=atoms.lattice_mat, length=dat["kpoint_length_unit"]
)
kpts = kp._kpoints[0]



Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699


100%|██████████| 40.8M/40.8M [00:02<00:00, 19.1MiB/s]


Loading the zipfile...
Loading completed.


In [8]:
kpts

[17, 17, 17]

In [9]:
ase_atoms

Atoms(symbols='Al4', pbc=True, cell=[4.06741, 4.06741, 4.06741])

In [10]:
# Sample calculation from https://wiki.fysik.dtu.dk/gpaw/tutorialsexercises/structureoptimization/aluminium/aluminium.html
from ase import Atoms
from ase.visualize import view
from gpaw import GPAW, PW



calc = GPAW(mode=PW(500),       # cutoff
            kpts=kpts,     # k-points
            txt=name + '.txt')  # output file

ase_atoms.calc = calc

energy = ase_atoms.get_potential_energy()
#calc.write(name + '.gpw')
print('Energy:', energy, 'eV')

Energy: -16.655990977510008 eV


In [ ]:
import numpy as np
from ase.eos import calculate_eos
dx=np.arange(-0.05, 0.05, 0.01)
eos = calculate_eos(ase_atoms)
v, e, B = eos.fit()


In [ ]:
from ase.units import kJ
kv = B / kJ * 1.0e24
kv

83.74848683247903

A quick calculation of Al(111)-Al2O3(001) interface energy

In [ ]:
import os
import numpy as np
from jarvis.analysis.interface.zur import make_interface
from jarvis.analysis.defects.surface import Surface
from gpaw import GPAW, PW

def jarvis_atoms_to_energy(atoms=None,kp=[2,2,1],xc='PBE',ecut=300):
       name='jarvis_db_gpaw'
       calc = GPAW(mode=PW(ecut),       # cutoff
            kpts=kp,     # k-points
            txt=name + '.txt')
       ase_atoms=atoms.ase_converter()
       ase_atoms.calc=calc
       atoms_en = ase_atoms.get_potential_energy()
       return atoms_en

def get_interface_energy(
    film_atoms=None,
    subs_atoms=None,
    film_index=[1, 1, 1],
    subs_index=[0, 0, 1],
    film_thickness=25,
    subs_thickness=25,
    model_path="",
    seperation=3.0,
    vacuum=8.0,
    max_area_ratio_tol=1.00,
    max_area=500,
    ltol=0.05,
    atol=1,
    apply_strain=False,
    from_conventional_structure=True,
):
    film_surf = Surface(
        film_atoms,
        indices=film_index,
        from_conventional_structure=from_conventional_structure,
        thickness=film_thickness,
        vacuum=vacuum,
    ).make_surface()
    subs_surf = Surface(
        subs_atoms,
        indices=subs_index,
        from_conventional_structure=from_conventional_structure,
        thickness=subs_thickness,
        vacuum=vacuum,
    ).make_surface()
    het = make_interface(
        film=film_surf,
        subs=subs_surf,
        seperation=seperation,
        vacuum=vacuum,
        max_area_ratio_tol=max_area_ratio_tol,
        max_area=max_area,
        ltol=ltol,
        atol=atol,
        apply_strain=apply_strain,
    )
    film_en = jarvis_atoms_to_energy(het["film_sl"])
    print('film_en', film_en)
    subs_en = jarvis_atoms_to_energy(het["subs_sl"])
    print ('subs_en', subs_en)
    intf_en = jarvis_atoms_to_energy(het["interface"])
    print('intf_en', intf_en)

    m = het["interface"].lattice.matrix
    area = np.linalg.norm(np.cross(m[0], m[1]))
    intf_energy = 16 * (intf_en - subs_en - film_en) / (area)  # J/m2
    het["interface_energy"] = intf_energy
    return het




It might take more than a hour

In [ ]:
from jarvis.db.figshare import get_jid_data
from jarvis.core.atoms import Atoms
atoms_al = Atoms.from_dict(
    get_jid_data(dataset="dft_3d", jid="JVASP-816")["atoms"]
)
atoms_al2o3 = Atoms.from_dict(
    get_jid_data(dataset="dft_3d", jid="JVASP-32")["atoms"]
)
intf = get_interface_energy(
    film_atoms=atoms_al, subs_atoms=atoms_al2o3, film_thickness=10, subs_thickness=10
)
print(intf["interface_energy"] )


Obtaining 3D dataset 55k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Loading the zipfile...
Loading completed.
Obtaining 3D dataset 55k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Loading the zipfile...
Loading completed.
mismatch_u,mismatch_v 0.03706886202229698 0.03706805926123646


/usr/local/lib/python3.7/site-packages/jarvis/core/atoms.py:1596: RuntimeWarning: divide by zero encountered in double_scalars
  ) / bottom.lattice_mat[1][1]


film_en -140.35238830139863
subs_en -440.90161284833647
intf_en -587.6015482599141
-1.2706412219766434
